

*   Установка библиотек



In [ ]:
!pip install transformers accelerate peft datasets sentencepiece tqdm bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

Подключаем google диск

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
if torch.cuda.is_available():
    print("GPU доступен!")
    print("Имя устройства GPU:", torch.cuda.get_device_name(0))
    print("Количество GPU:", torch.cuda.device_count())
    # print("Выделенная память (МБ):", torch.cuda.memory_allocated() / (1024**2)) # Может вызвать ошибку, если нет выделенной памяти
    # print("Кэшированная память (МБ):", torch.cuda.memory_cached() / (1024**2)) # Может вызвать ошибку
else:
    print("GPU недоступен. Убедитесь, что в настройках среды выполнения выбран 'GPU'.")

GPU доступен!
Имя устройства GPU: Tesla T4
Количество GPU: 1


Импорт необходимых библиотек

In [ ]:
import os
import torch
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training # Добавлен prepare_model_for_kbit_training
from transformers import BitsAndBytesConfig # Добавлен импорт для 4-битного квантования
import json
import logging
from tqdm.auto import tqdm # Для прогресс-баров

Настроим логирование

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

Определение констант

In [ ]:
MODEL_NAME = "Qwen/Qwen3-1.7B" # Используем Qwen3-1.7B
DATASET_PATH = "/content/drive/MyDrive/qwen3_adaptive_reasoning_dataset.jsonl"
OUTPUT_DIR = "/content/drive/MyDrive/qwen3_1_7b_reasoning_finetuned_gpu"

Параметры LoRA

In [ ]:
LORA_R = 32 # Увеличено для лучшей способности к обучению
LORA_ALPHA = 64 # Увеличено вместе с LORA_R
LORA_DROPOUT = 0.05
# Целевые модули для LoRA (все основные линейные слои для Qwen)
TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]

Гиперпараметры для дообучения

In [ ]:
LEARNING_RATE = 2e-5
# BATCH_SIZE и GRADIENT_ACCUMULATION_STEPS для GPU
# Эффективный батч = BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS
BATCH_SIZE = 1
GRADIENT_ACCUMULATION_STEPS = 8 # Эффективный батч = 1 * 8 = 8
NUM_TRAIN_EPOCHS = 1
SAVE_STEPS = 50
LOGGING_STEPS = 50
MAX_SEQ_LEN = 1024 # Максимальная длина последовательности


Основной код для обучения

In [ ]:
# --- ОТКЛЮЧАЕМ КВАНТИЗАЦИЮ ---
USE_4BIT_QUANTIZATION = False

# --- ОСНОВНАЯ ФУНКЦИЯ ОБУЧЕНИЯ ---
def main_training_gpu():
    if not torch.cuda.is_available():
        logger.error("GPU не обнаружен. Убедитесь, что в настройках среды выполнения выбран 'GPU'.")
        return

    logger.info(f"Загрузка токенайзера и модели: {MODEL_NAME}")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.padding_side = "right" # Qwen1.5-7B-Chat использует padding_side="right"

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto", # Автоматически загружает на GPU
        torch_dtype=torch.bfloat16, # Используем bfloat16 для лучшей точности и стабильности на мощных GPU
        trust_remote_code=True,
    )
    logger.info("Модель загружена в режиме полной точности (bfloat16 на GPU).")

    # Настройка LoRA
    lora_config = LoraConfig(
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
        lora_dropout=LORA_DROPOUT,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=TARGET_MODULES
    )
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()

    # --- ПОДГОТОВКА ДАТАСЕТА ---
    logger.info(f"Загрузка и подготовка датасета из {DATASET_PATH}")

    def load_jsonl_dataset(file_path):
        """Загружает JSONL файл и преобразует его в Hugging Face Dataset."""
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Файл датасета не найден по пути: {file_path}. Убедитесь, что Google Drive смонтирован и путь корректен.")

        data = []
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                data.append(json.loads(line))
        return Dataset.from_list(data)

    raw_dataset = load_jsonl_dataset(DATASET_PATH)

    def preprocess_function(examples):
        formatted_texts = []
        for messages in examples['messages']:
            text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
            formatted_texts.append(text)

        tokenized_inputs = tokenizer(
            formatted_texts,
            truncation=True,
            max_length=MAX_SEQ_LEN,
            padding="max_length", # Паддинг до MAX_SEQ_LEN
            return_tensors="pt", # Возвращаем PyTorch тензоры
        )
        # Для вычисления потери (perplexity) метки должны быть входными ID, смещенными на 1
        tokenized_inputs["labels"] = tokenized_inputs["input_ids"].clone()
        return tokenized_inputs

    processed_dataset = raw_dataset.map(
        preprocess_function,
        batched=True,
        num_proc=1, # Всегда 1 для Colab, чтобы избежать проблем с процессами/памятью
        remove_columns=raw_dataset.column_names
    )

    train_test_split = processed_dataset.train_test_split(test_size=0.05, seed=42)
    train_dataset = train_test_split['train']
    eval_dataset = train_test_split['test']

    logger.info(f"Общий размер тренировочного датасета: {len(train_dataset)}")
    logger.info(f"Общий размер валидационного датасета: {len(eval_dataset)}")

    # --- НАСТРОЙКА АРГУМЕНТОВ ОБУЧЕНИЯ И ТРЕНЕРА ---
    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        per_device_train_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        learning_rate=LEARNING_RATE,
        num_train_epochs=NUM_TRAIN_EPOCHS,
        logging_steps=LOGGING_STEPS,
        save_strategy="steps",
        save_steps=SAVE_STEPS,
        eval_strategy="steps",
        eval_steps=SAVE_STEPS,
        fp16=True,
        bf16=False,
        push_to_hub=False,
        report_to="none",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        save_total_limit=2,
        dataloader_num_workers=0, # Для Colab часто 0
        optim="adamw_torch_8bit", # Используем 8-битный оптимизатор для экономии RAM, не влияет на точность модели
        per_device_eval_batch_size=BATCH_SIZE # Используем тот же размер батча для оценки
    )

    data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    # Запуск обучения
    logger.info("Запуск дообучения на GPU...")
    trainer.train()

    # Сохранение модели
    logger.info("Обучение завершено. Сохранение финальной модели LoRA адаптеров...")
    trainer.save_model(OUTPUT_DIR)
    logger.info(f"Адаптеры LoRA сохранены в: {OUTPUT_DIR}")

    # --- Слияние модели и инференс ---
    try:
        from peft import PeftModel
        logger.info("Слияние LoRA адаптеров и сохранение полной модели (может потребоваться много RAM)...")
        base_model_for_merge = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            device_map="auto",
            torch_dtype=torch.bfloat16,
            trust_remote_code=True
        )
        peft_model_for_merge = PeftModel.from_pretrained(base_model_for_merge, OUTPUT_DIR)
        merged_model = peft_model_for_merge.merge_and_unload()

        merged_model_path = os.path.join(OUTPUT_DIR, "merged_model")
        merged_model.save_pretrained(merged_model_path)
        tokenizer.save_pretrained(merged_model_path)
        logger.info(f"Полная дообученная модель сохранена в: {merged_model_path}")
    except Exception as e:
        logger.warning(f"Не удалось слить и сохранить полную модель (возможно, не хватило RAM): {e}")
        logger.info(f"Адаптеры LoRA сохранены в: {OUTPUT_DIR}. Вы можете использовать их с оригинальной моделью {MODEL_NAME}.")

    logger.info("\n--- Пример использования дообученной модели ---")
    try:
        # Загружаем модель для инференса (можно загрузить только адаптеры, если RAM ограничена)
        # Или загрузить слитую модель, если она была сохранена
        if os.path.exists(os.path.join(OUTPUT_DIR, "merged_model")):
            inference_model = AutoModelForCausalLM.from_pretrained(
                os.path.join(OUTPUT_DIR, "merged_model"),
                device_map="auto",
                torch_dtype=torch.bfloat16, # Используем bfloat16 для инференса
                trust_remote_code=True
            )
            inference_tokenizer = AutoTokenizer.from_pretrained(os.path.join(OUTPUT_DIR, "merged_model"), trust_remote_code=True)
        else:
            # Если слитая модель не сохранилась, используем базовую + адаптеры
            base_model_for_inference = AutoModelForCausalLM.from_pretrained(
                MODEL_NAME,
                device_map="auto",
                torch_dtype=torch.bfloat16, # Используем bfloat16 для инференса
                trust_remote_code=True
            )
            inference_model = PeftModel.from_pretrained(base_model_for_inference, OUTPUT_DIR)
            inference_tokenizer = tokenizer # Используем тот же токенайзер

        inference_model.eval()

        test_question = "Какое число является следующим в последовательности 1, 1, 2, 3, 5, 8, 13?"

        test_messages = [
            {"role": "user", "content": test_question}
        ]
        formatted_input = inference_tokenizer.apply_chat_template(
            test_messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt" # Возвращаем PyTorch тензоры
        )

        if torch.cuda.is_available():
            formatted_input = formatted_input.to("cuda")
            inference_model.to("cuda")

        logger.info(f"\nВопрос: {test_question}")
        logger.info("Генерация ответа (может занять некоторое время)...")

        with torch.no_grad():
            output_tokens = inference_model.generate(
                formatted_input,
                max_new_tokens=512,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                repetition_penalty=1.1,
                eos_token_id=inference_tokenizer.eos_token_id,
                pad_token_id=inference_tokenizer.pad_token_id
            )

        generated_text = inference_tokenizer.decode(output_tokens[0][formatted_input.shape[1]:], skip_special_tokens=True)
        logger.info(f"Сгенерированный ответ:\n{generated_text}")

    except Exception as e:
        logger.error(f"Ошибка при попытке загрузки или использования дообученной модели: {e}")
        logger.info("Убедитесь, что модель успешно дообучилась и адаптеры сохранены.")


if __name__ == "__main__":
    main_training_gpu()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

trainable params: 34,865,152 || all params: 1,755,440,128 || trainable%: 1.9861


Map:   0%|          | 0/5934 [00:00<?, ? examples/s]

/tmp/ipython-input-9-3570740250.py:112: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss
50,1.026400,0.838173
100,0.781500,0.759293
150,0.745400,0.738568
200,0.728100,0.725385
250,0.716600,0.716590
300,0.713600,0.709102
350,0.702700,0.702960
400,0.697200,0.698468
450,0.683100,0.694378
500,0.696800,0.691590


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Функция для работы с tools

In [ ]:
import subprocess
import re
import torch
import logging
import os
# --- Настройка логирования, если еще не настроено ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- Константы ---
OUTPUT_DIR = "/content/drive/MyDrive/qwen3_1_7b_reasoning_finetuned_gpu"
MODEL_NAME = "Qwen/Qwen3-1.7B"
# Переменная tokenizer может быть не определена в этом скоупе,
# ее нужно будет загрузить вместе с моделью.

def execute_python_code(code: str) -> str:
    """
    Безопасно выполняет код Python в отдельном процессе и возвращает его stdout.
    ВАЖНО: Этот метод уязвим. Для реальных задач используйте полноценную песочницу (sandbox).
    """
    try:
        code = code.strip()
        result = subprocess.run(
            ['python', '-c', code],
            capture_output=True,
            text=True,
            timeout=15,
            check=False
        )
        if result.returncode != 0:
            return f"Ошибка выполнения:\n{result.stderr}"
        output = result.stdout.strip()
        return output if output else "Код выполнен успешно, но ничего не вывел."
    except subprocess.TimeoutExpired:
        return "Ошибка: Время выполнения кода истекло."
    except Exception as e:
        return f"Критическая ошибка при попытке выполнить код: {e}"

def run_inference_with_code_execution(
    inference_model,
    inference_tokenizer,
    question: str,
    max_loops=5
):
    """
    Запускает цикл инференса с выполнением кода из Markdown-блоков.
    """
    messages = [{"role": "user", "content": question}]

    logger.info(f"\nВопрос: {question}")
    logger.info("Начинаю генерацию с выполнением кода...")

    loop_count = 0
    while loop_count < max_loops:
        loop_count += 1
        logger.info(f"--- Итерация {loop_count} ---")

        prompt_text = inference_tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        inputs = inference_tokenizer(prompt_text, return_tensors="pt").to(inference_model.device)

        with torch.no_grad():
            output_tokens = inference_model.generate(
                **inputs,
                max_new_tokens=512,
                do_sample=True,
                temperature=0.2,
                top_p=0.9,
                repetition_penalty=1.1,
                eos_token_id=[
                    inference_tokenizer.eos_token_id,
                    inference_tokenizer.convert_tokens_to_ids('`')
                ],
                pad_token_id=inference_tokenizer.pad_token_id
            )

        new_text = inference_tokenizer.decode(output_tokens[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)

        code_to_execute = None
        match = re.search(r'```python(.*?)```', new_text, re.DOTALL)

        if match:
            code_to_execute = match.group(1)
            reasoning_part = new_text.split('```python')[0]
            messages.append({"role": "assistant", "content": reasoning_part})
            logger.info(f"Модель сгенерировала рассуждение:\n{reasoning_part}")
        else:
            logger.info("Модель сгенерировала финальный ответ.")
            messages.append({"role": "assistant", "content": new_text})
            break

        if code_to_execute:
            logger.info(f"Обнаружен код для выполнения:\n---\n{code_to_execute.strip()}\n---")
            execution_result = execute_python_code(code_to_execute)
            logger.info(f"Результат выполнения:\n{execution_result}")

            # Формируем ответ от "инструмента" и добавляем в историю
            tool_response = f"Результат выполнения кода:\n```\n{execution_result}\n```"
            messages.append({"role": "user", "content": tool_response})

    final_answer = messages[-1]['content']
    logger.info(f"\n✅ Итоговый сгенерированный ответ:\n{final_answer}")
    return final_answer

# --- Блок для запуска ---
print("\n--- Пример использования (без изменения датасета) ---")
try:
      # Загрузка токенайзера
      # Если переменная tokenizer не определена, загрузите ее здесь
      try:
          tokenizer
      except NameError:
          tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

      if os.path.exists(os.path.join(OUTPUT_DIR, "merged_model")):
          inference_model = AutoModelForCausalLM.from_pretrained(
              os.path.join(OUTPUT_DIR, "merged_model"), device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True
          )
          inference_tokenizer = AutoTokenizer.from_pretrained(os.path.join(OUTPUT_DIR, "merged_model"), trust_remote_code=True)
      else:
          base_model_for_inference = AutoModelForCausalLM.from_pretrained(
              MODEL_NAME, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True
          )
          inference_model = PeftModel.from_pretrained(base_model_for_inference, OUTPUT_DIR)
          inference_tokenizer = tokenizer

      inference_model.eval()

      test_question = "Посчитай сумму всех чисел от 1 до 100."

      run_inference_with_code_execution(
          inference_model,
          inference_tokenizer,
          test_question
      )
except Exception as e:
      logger.error(f"Ошибка: {e}", exc_info=True)
      print("Убедитесь, что модель успешно дообучилась и адаптеры сохранены.")


--- Пример использования (без изменения датасета) ---


In [ ]:
test_question = "В магазине было 150 кг яблок. В первый день продали 30% всех яблок, а во второй день - 45% от того, что продали в первый день. Сколько кг яблок осталось в магазине?"

run_inference_with_code_execution(
    inference_model,
    inference_tokenizer,
    test_question
)

'<think>\n<chunk_start>\nСначала определим сколько яблок продали в первый день: это 30% от общего количества (150 кг).\n</chunk_end><continue_thinking>\n<chunk_start>\nВычислим 30% от 150 кг: 150 × 0.3 = 45 кг.\n</chunk_end><continue_thinking>\n<chunk_start>\nОставшиеся после первого дня яблоки составляют 150 − 45 = 105 кг.\n</chunk_end><continue_thinking>\n<chunk_start>\nТеперь рассчитаем, сколько яблок продали во второй день — это 45% от того, что продали в первый день.\n</chunk_end><continue_thinking>\n<chunk_start>\nВычислим 45% от 45 кг: 45 × 0.45 = 20.25 кг.\n</chunk_end><continue_thinking>\n<chunk_start>\nИтак, во второй день продали 20.25 кг яблок. Теперь вычтем их из остатка на втором этапе.\n</chunk_end><continue_thinking>\n<chunk_start>\nОставшееся количество яблок: 105 − 20.25 = 84.75 кг.\n</chunk_end><end_of_thought>\n</think>\n\n84.75'